# Comparisions

This notebook contains the code to reproduce the comparisons between:
1) plasma and csf findings from this study
2) findings from this study with other studies

In [ ]:
import pandas as pd
import numpy as np
import os

import plotly.express as px
from upsetplot import plot
from matplotlib import pyplot
import seaborn as sns
import plotly.graph_objects as go
import random
import plotly.io as pio
from upsetplot import plot
from matplotlib import pyplot

import alphastats

In [ ]:
# load results from this study
nielsen_CSF_uniprot = pd.read_csv("../submission/combined_lst_csf_pval.csv")
proteome_raw = pd.read_csv('../data/CSFreport.pg_matrix.tsv', sep = '\t')
nielsen_CSF = proteome_raw[proteome_raw['Protein.Group'].isin(nielsen_CSF_uniprot['csf_identifiers_pval'].tolist())][['Protein.Group','Genes']]
#nielsen_CSF['Protein.Group'] = nielsen_CSF['Protein.Group'].str.split(';').str[0]
#nielsen_CSF['Genes'] = nielsen_CSF['Genes'].str.split(';').str[0]

nielsen_CSF_uniprot_log2 = pd.read_csv("../submission/combined_lst_csf.csv")
nielsen_CSF_log2 = proteome_raw[proteome_raw['Protein.Group'].isin(nielsen_CSF_uniprot_log2['csf_identifiers'].tolist())][['Protein.Group','Genes']]
#nielsen_CSF_log2['Protein.Group'] = nielsen_CSF_log2['Protein.Group'].str.split(';').str[0]
#nielsen_CSF_log2['Genes'] = nielsen_CSF['Genes'].str.split(';').str[0]

nielsen_plasma_uniprot = pd.read_csv("../submission/combined_lst_plasma.csv")
proteome_raw = pd.read_csv('../data/plasmareport.pg_matrix.tsv', sep = '\t')
nielsen_plasma = proteome_raw[proteome_raw['Protein.Group'].isin(nielsen_plasma_uniprot['plasma_identifiers'].tolist())][['Protein.Group','Genes']]
#nielsen_plasma['Protein.Group'] = nielsen_plasma['Protein.Group'].str.split(';').str[0]
#nielsen_plasma['Genes'] = nielsen_plasma['Genes'].str.split(';').str[0]


In [ ]:
plt.figure(figsize=(6, 6))
venn3(subsets = [set(nielsen_plasma['Protein.Group'].unique().tolist()), set(nielsen_CSF_log2['Protein.Group'].unique().tolist()), set(nielsen_CSF['Protein.Group'].unique().tolist())], set_labels=('plasma','CSF_pval','CSF_PVAL_LOG2'))
combined_blood_csf = [x for x in nielsen_plasma['Protein.Group'].unique().tolist() if x in nielsen_CSF['Protein.Group'].unique().tolist()]
print(len(combined_blood_csf))
#pd.DataFrame(combined_blood_csf, columns = ['identifiers']).to_csv('../submission/combined_blood_csf.csv', index = None)


#plt.savefig("../output_blood/venn_diagram_CSF_plasma_pval_log2.svg", format="svg")
plt.show()



In [ ]:
dfs = []
for groups in [['csf','development'],['plasma','development']]:
    tissue = groups[0]
    run = groups[1]
    if tissue == 'csf':
        loader = alphastats.DIANNLoader(file = "../submission/aps_data_corrected_csf_{}.tsv".format(run))
    else:
        loader = alphastats.DIANNLoader(file = "../submission/aps_data_corrected_{}_{}.tsv".format(tissue, run))
    dataset = alphastats.DataSet(
        loader = loader, 
        metadata_path="../submission/aps_meta_{}.xlsx".format(tissue), 
        sample_column="sample_id"
    )
    dataset.preprocess(
        log2_transform=True,
        remove_contaminations=True
    )

In [ ]:
# is up/down regulation in lnb the same direction for plasma and csf?

dfs = []
for groups in [['csf','development'],['plasma','development']]:
    print(groups)
    tissue = groups[0]
    run = groups[1]
    if tissue == 'csf':
        loader = alphastats.DIANNLoader(file = "../submission/aps_data_corrected_csf_{}.tsv".format(run))
    else:
        loader = alphastats.DIANNLoader(file = "../submission/aps_data_corrected_{}_{}.tsv".format(tissue, run))
    dataset = alphastats.DataSet(
        loader = loader, 
        metadata_path="../submission/aps_meta_{}.xlsx".format(tissue), 
        sample_column="sample_id"
    )
    dataset.preprocess(
        log2_transform=True,
        remove_contaminations=True
    )
    diff_exp_VM = dataset.diff_expression_analysis('LNB','VM',column='diagnosis', fdr=0.05)
    diff_exp_VM = diff_exp_VM.set_index(['Protein.Group']).T[combined_blood_csf]
    diff_exp_VM['comparison'] = 'VM'
    diff_exp_VM['tissue'] = tissue
    diff_exp_control = dataset.diff_expression_analysis('LNB','control',column='diagnosis', fdr=0.05)
    diff_exp_control= diff_exp_control.set_index(['Protein.Group']).T[combined_blood_csf]
    diff_exp_control['comparison'] = 'control'
    diff_exp_control['tissue'] = tissue
    dfs.append(diff_exp_VM)
    dfs.append(diff_exp_control)



In [ ]:
dfs_df = pd.concat(dfs).reset_index().set_index(['index','comparison','tissue'])
# add gene
gene_mapping = proteome_raw.set_index('Protein.Group')
# replace missing genenames with protein identifier
gene_mapping.loc[gene_mapping.Genes.isnull(), "Genes"]=gene_mapping.loc[gene_mapping.Genes.isnull()].index.tolist()
dfs_df.columns = gene_mapping.loc[combined_blood_csf]['Genes'].tolist()


In [ ]:
sub_dfs = dfs_df.loc[dfs_df.index.get_level_values('index') == 'log2fc'].droplevel('index')
#order = sub_dfs.mean().sort_values().index.tolist()
sub_dfs = sub_dfs.T
sub_null = dfs_df.loc[dfs_df.index.get_level_values('index') == 'pval'].droplevel('index')
sub_null = sub_null.T
sub_null[sub_null>=0.05] = np.nan

g = sns.clustermap(sub_dfs, mask = sub_null.isnull(), 
               cmap = 'vlag', 
               center = 0,
               figsize = (3,20),
               col_cluster=False, row_cluster=True)
g.figure.savefig("../submission/f5b.svg")

In [ ]:
heatmap_order = sub_dfs.iloc[g.dendrogram_row.reordered_ind].index.tolist()

In [ ]:
# Sankey data
annotation_table = pd.read_csv('../data/annotation_table_plasma.tsv', sep = '\t')

sankey_df = pd.merge(pd.DataFrame(gene_mapping.reset_index().set_index('Genes').loc[sub_dfs.index.tolist()]['Protein.Group'].tolist(), columns = ['Entry']), annotation_table[['Entry','GO','Gene Names']].drop_duplicates(), on = 'Entry', how = 'left')
sankey_df.fillna('nan', inplace = True)
# sort the df according to heatmap
sankey_df['Entry'] = pd.Categorical(sankey_df['Entry'], categories=heatmap_order, ordered=True)
sankey_df = sankey_df.sort_values(['Entry'])
# format the df
sankey_df.Entry = sankey_df.Entry.astype(str)
sankey_df['Gene Names'] = sankey_df['Gene Names'].str.split(' ', expand = True)[0] 

In [ ]:
# Sankey plot

# Get unique values of Genes and annotations
all_Genes = list(sankey_df['Gene Names'].unique())
#all_Genes.reverse()
all_annotations = list(sankey_df['GO'].unique())


# Create indices for sankey source (Gene Namess) and targets (annotations)
sankey_df['Gene Names_index'] = sankey_df['Gene Names'].apply(lambda x: all_Genes.index(x))
sankey_df = sankey_df.sort_values(by='Gene Names_index').reset_index(drop=True)  # Ensure sankey_df is sorted by the order of 'all_Genes'
sankey_df['annotation_index'] = sankey_df['GO'].apply(lambda x: len(all_Genes) + all_annotations.index(x))

# Generate a color for each annotation
colors = ['#'+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(len(all_annotations))]

# Map annotation index to colors
target_colors = {len(all_Genes) + i: colors[i] for i in range(len(all_annotations))}

# Create a list of colors for the links based on target node
link_colors = [target_colors[target] for target in sankey_df['annotation_index']]

# Define node positions for source nodes (x = 0.1 for all sources, y spaced evenly)
source_y_positions = [i / len(all_Genes) for i in range(len(all_Genes))]  # Evenly spaced source y positions
source_x_positions = [0.1] * len(all_Genes)  # Fixed x position for sources

# Only set positions for sources, leave target positions to be automatically placed
node_x_positions = source_x_positions + [None] * len(all_annotations)  # No x positions for targets
node_y_positions = source_y_positions + [None] * len(all_annotations)  # No y positions for targets

# Create Sankey diagram
fig = go.Figure(go.Sankey(
    node = dict(
        pad = 15,
        thickness = 20,
        line = dict(color = "black", width = 0.5),
        label = all_Genes + all_annotations,
        x = node_x_positions,
        y = node_y_positions
    ),
    link = dict(
        source = sankey_df['Gene Names_index'],  # starting points (Gene Namess)
        target = sankey_df['annotation_index'],  # end points (annotations)
        value = [1] * len(sankey_df),  # each link has equal value (1 for each protein-GO link)
        color = link_colors  # set the color of links
    )
))

# Update layout and display the figure
fig.update_layout(title_text="Sankey Diagram of Protein Gene Names and Annotations", 
                  font_size=10,
                  height=800,
                  width=600)

# Define the path where you want to save the PDF
output_path = '../submission/f5b_sankey.svg'

# Save the figure 
pio.write_image(fig, output_path, format='svg')

fig.show()

### Comparison to other studies

In [ ]:
# This study
nielsen_CSF['Protein.Group'] = nielsen_CSF['Protein.Group'].str.split(';').str[0]
nielsen_CSF['Genes'] = nielsen_CSF['Genes'].str.split(';').str[0]

nielsen_CSF_log2['Protein.Group'] = nielsen_CSF_log2['Protein.Group'].str.split(';').str[0]
nielsen_CSF_log2['Genes'] = nielsen_CSF['Genes'].str.split(';').str[0]

nielsen_plasma['Protein.Group'] = nielsen_plasma['Protein.Group'].str.split(';').str[0]
nielsen_plasma['Genes'] = nielsen_plasma['Genes'].str.split(';').str[0]

In [ ]:
# load gene to uniprot_id mapping (downloaded from uniprot)
idmapping = pd.read_csv('idmapping.tsv', sep='\t')
idmapping.columns = ['Protein.Group','Genes']
idmapping['Genes'].unique().shape
#NOTE some genes have multiple uniprot id mappings: idmapping[idmapping.Genes.duplicated(keep = False)].sort_values(['Genes'])



In [ ]:
# Study by Angel et al
angel = pd.read_excel('angel.xlsx')
angel_ms_csf_LD = angel[angel['p-value'] < 0.05]['Gene Symbol'].str.split(';').str[0].unique().tolist()
angel_ms_csf_LD_uniprot = idmapping[idmapping.Genes.isin(angel_ms_csf_LD)]['Protein.Group'].unique().tolist()
print(len(angel_ms_csf_LD))
print(len(angel_ms_csf_LD_uniprot))


In [ ]:
additional_mapping = [item for item in angel_ms_csf_LD if item not in idmapping['Genes'].tolist()]
angel_ms_csf_LD_uniprot += additional_mapping

In [ ]:
#Fredriksson et al
fredriksson_olink_plasma_children = ['CST5', 'IL15RA', 'CXCL10', 'DNER', 'CX3CL1']
fredriksson_olink_plasma_children_uniprot = idmapping[idmapping.Genes.isin(fredriksson_olink_plasma_children)]['Protein.Group'].unique().tolist()
fredriksson_olink_plasma_children_uniprot

In [ ]:
# Nilsson et al

# CSF
nilsson_olink_csf_PTLDS = pd.read_excel('nilsson.xlsx', sheet_name = 'T-test CSF')
nilsson_olink_csf_PTLDS_gene = nilsson_olink_csf_PTLDS[nilsson_olink_csf_PTLDS['Adjusted_pval']< 0.05]['Name'].unique().tolist()
nilsson_olink_csf_PTLDS_uniprot = nilsson_olink_csf_PTLDS[nilsson_olink_csf_PTLDS['Adjusted_pval']< 0.05]['UniProt'].str.split(',').str[0].unique().tolist()
print(len(nilsson_olink_csf_PTLDS_gene))
print(len(nilsson_olink_csf_PTLDS_uniprot))
# Serum
nilsson_olink_serum_PTLDS =  pd.read_excel('nilsson.xlsx', sheet_name = 'T-test Serum')
nilsson_olink_serum_PTLDS_gene = nilsson_olink_serum_PTLDS[nilsson_olink_serum_PTLDS['Adjusted_pval']< 0.05]['Name'].unique().tolist()
nilsson_olink_serum_PTLDS_uniprot = nilsson_olink_serum_PTLDS[nilsson_olink_serum_PTLDS['Adjusted_pval']< 0.05]['UniProt'].str.split(',').str[0].unique().tolist()
print(len(nilsson_olink_serum_PTLDS_gene))
print(len(nilsson_olink_serum_PTLDS_uniprot))


In [ ]:
# Gegotek et al
Gegotek_ms_serum_NB = pd.read_excel('gegotek/1-s2.0-S0882401024005618-mmc1.xlsx')
Gegotek_ms_serum_NB = Gegotek_ms_serum_NB[Gegotek_ms_serum_NB.comparison.str.contains('NB1_ctl')]
Gegotek_ms_serum_NB_uniprot = Gegotek_ms_serum_NB[Gegotek_ms_serum_NB['p.val'] < 0.05]['Protein ID'].unique().tolist()
print(len(Gegotek_ms_serum_NB[Gegotek_ms_serum_NB['p.val'] < 0.05]['Protein ID'].unique().tolist()))

In [ ]:
nielsen_CSF_uniprot = nielsen_CSF['Protein.Group'].unique().tolist()
print(len(nielsen_CSF))
nielsen_plasma_uniprot = nielsen_plasma['Protein.Group'].unique().tolist()
#nielsen_plasma = pd.read_excel('nielsen_plasma.xlsx')
#nielsen_plasma_uniprot = nielsen_plasma[nielsen_plasma['Adjusted p-value'] < 0.05]['Protein.Group'].str.split(';').str[0].unique().tolist() 
#nielsen_plasma = nielsen_plasma[nielsen_plasma['Adjusted p-value'] < 0.05]['Genes'].str.split(';').str[0].unique().tolist()
print(len(nielsen_plasma_uniprot))

In [ ]:
# gather studies

cols = [
    pd.Series(nielsen_CSF_uniprot, index=nielsen_CSF_uniprot, name="nielsen_CSF_uniprot"),
    pd.Series(nielsen_plasma_uniprot, index=nielsen_plasma_uniprot, name="nielsen_plasma_uniprot"),
    pd.Series(angel_ms_csf_LD_uniprot, index=angel_ms_csf_LD_uniprot, name="angel_ms_csf_LD_uniprot"),
    pd.Series(Gegotek_ms_serum_NB_uniprot, index=Gegotek_ms_serum_NB_uniprot, name="Gegotek_ms_serum_NB_uniprot"),
    pd.Series(nilsson_olink_csf_PTLDS_uniprot, index=nilsson_olink_csf_PTLDS_uniprot, name="nilsson_olink_csf_PTLDS_uniprot"),
    pd.Series(nilsson_olink_serum_PTLDS_uniprot, index=nilsson_olink_serum_PTLDS_uniprot, name="nilsson_olink_serum_PTLDS_uniprot"),
    pd.Series(fredriksson_olink_plasma_children_uniprot, index=fredriksson_olink_plasma_children_uniprot, name="fredriksson_olink_plasma_children_uniprot"),
]

df = pd.concat(cols, axis=1).sort_index()


In [ ]:
df = pd.merge(idmapping, df, right_index=True, left_on='Protein.Group', how='right').reset_index(drop=True)
# name genes with no mapping 'Unknown' + cumulative sum
df.loc[df.Genes.isnull(),'Genes'] = 'Unknown_' + (df.Genes.isnull().cumsum()).astype(str)
df[df.Genes.str.contains('Unknown')]


In [ ]:
# Handle duplicated genes
# Identify study columns (everything except Protein.Group + Genes)
study_cols = df.columns.difference(["Protein.Group", "Genes"])

# Group by Genes
collapsed = (
    df.groupby("Genes", as_index=False)
      .agg(
          {
              "Protein.Group": lambda x: ";".join(x),   # merge all protein IDs
              **{col: lambda x: ";".join(sorted(set(x.dropna()))) for col in study_cols}
          }
      )
)
collapsed.replace('', np.nan, inplace=True)
collapsed = collapsed.drop_duplicates()
print(df.shape)
print(collapsed.shape)

In [ ]:
df_overlap = df.set_index(['Genes', 'Protein.Group']).dropna(thresh=2)
# subset to this study
df_overlap = df_overlap[(df_overlap.nielsen_CSF_uniprot.notnull())|(df_overlap.nielsen_plasma_uniprot.notnull())].sort_values(['Genes'])
df_overlap[['nielsen_CSF_uniprot','nielsen_plasma_uniprot','angel_ms_csf_LD_uniprot','Gegotek_ms_serum_NB_uniprot','nilsson_olink_csf_PTLDS_uniprot','nilsson_olink_serum_PTLDS_uniprot','fredriksson_olink_plasma_children_uniprot']].to_excel('comparison_proteins_overlap.xlsx')
df_overlap.shape

In [ ]:
pivot_df = df.set_index('Protein.Group').drop(columns=['Genes'])

# replace all valid values with 1
pivot_df = pivot_df.notnull().astype(int)

# Convert to boolean (if not already)
df_bool = pivot_df.astype(bool)

# Convert to a MultiIndex Series usable by upsetplot
upset_data = df_bool.groupby(list(df_bool.columns)).size()


In [ ]:
from upsetplot import UpSet

upset = UpSet(upset_data, show_counts=True)
upset.plot()
pyplot.savefig("../submission/f5c.pdf")